# Padise 2024
## Masinnägemise töötuba
Koostanud: Joonas Järve

Valige endale huvipakkuv ülesanne, mida lahendada ning probleemide korral guugeldage või küsige juhendajalt või kaasõpilastelt.


Erinevad ülesanded on erineva raskusastmega, kuid kõik jälgivad sama põhimõtet: 

1. ülesande lahendab **MUDEL** 
2. mudeli tööd demonstreerib **veebikaamera liides**

Siin on antud üldine veebikaamera liidese tsükkel ning sinna sisse tuleb tavaliselt panna mudel, mis töötleb pilte ja tagastab tulemuse veebikaamera väljundisse i.e ekraanile :) 

In [1]:
import cv2, sys, numpy, os 


cap = cv2.VideoCapture(0) #alustab veebikaamera

while True:
    ret, frame = cap.read() #loeb sisse kaadri kaamerast
    if not ret: #kontrollib et midagi ei läinud valesti
        break
    ##
    ##
    ##Siin saab teha kaadritöötlust ja lisada masinnägemise MUDELi 
    ##
    ##
    cv2.imshow('Väljund', frame) #kuvab pildi veebikaamera väljundisse

    # Programm lõpeb, kui vajutame 'q' 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Lõpetuseks vabastame kaamera
cap.release()
cv2.destroyAllWindows()

### 1. variant : Näotuvastus  
(kood on suures osas pärit siit : https://www.geeksforgeeks.org/face-detection-using-python-and-opencv-with-webcam/)

Järgnev koodilõik on andmetekogumiseks

In [14]:
haar_file = './naotuvastus/haarcascade_frontalface_default.xml' #mudeli asukoht
datasets = './naotuvastus/datasets' #näopiltide asukoht

sub_data = 'Minu Nimi'	# kaust minu näo piltidest (muuda vastavalt)

# Kontrolli, kas kaust on olemas; kui ei, siis loo see
path = os.path.join(datasets, sub_data) 
if not os.path.isdir(path): 
	os.mkdir(path) 

# pildi suurus
(width, height) = (130, 100)	 

#'0' on veebikaamera tüüpiliselt, 
# kui on probleeme proovi '1'
face_cascade = cv2.CascadeClassifier(haar_file) #laeme mudeli 

cap = cv2.VideoCapture(0)

# See koodijupp teeb veebikaameraga ühe pildi ja kasutab seda hiljem teie ära tundmiseks 
count = 1
#to-do1: teha see osa ümber nii, et programm koguks etteantud x arvu pilte 
#to-do2: nõua, et iga järgev pilt oleks eelnevast piisavalt erinev (kasutada nt. eukleidilist kaugust vms.)
# või tehakse mingi aja tagant

ret, frame = cap.read()
    # Muuda pilt halltoonideks paremaks näotuvastuseks
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
faces = face_cascade.detectMultiScale(gray, 1.3, 4) # Tuvasta näod pildil
for (x, y, w, h) in faces: 
	# Joonista ruut näo ümber
	cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2) 
	# Võta näo osa, muuda suurust ja salvesta
	face = gray[y:y + h, x:x + w] 
	face_resize = cv2.resize(face, (width, height)) 
	cv2.imwrite('% s/% s.png' % (path, count), face_resize) 


# Näita pilti koos näo ruuduga
cv2.imshow('OpenCV', frame) 

# Lõpetuseks vabastame kaamera
cap.release()
cv2.destroyAllWindows()


Teine osa treenib mudeli tehtud fotode põhjal ja tuvastab näo ja kuvab kui enesekindel mudel on

In [15]:

size = 4

# Tuvastame näo
print('Recognizing Face Please Be in sufficient Lights...') 

# Loome piltide järjendi ja vastavate nimede järjendi 
(images, labels, names, id) = ([], [], {}, 0) 
for (subdirs, dirs, files) in os.walk(datasets): 
	for subdir in dirs: 
		names[id] = subdir 
		subjectpath = os.path.join(datasets, subdir) 
		for filename in os.listdir(subjectpath): 
			path = subjectpath + '/' + filename 
			label = id
			images.append(cv2.imread(path, 0)) 
			labels.append(int(label)) 
		id += 1
(width, height) = (130, 100) 

# Create a Numpy array from the two lists above 
(images, labels) = [numpy.array(lis) for lis in [images, labels]] 

# OpenCV treenib kogutud fotode pealt mudeli
# NOTE FOR OpenCV2: remove '.face' 
model = cv2.face.LBPHFaceRecognizer_create() 
model.train(images, labels) 

# Kasutame fisherRecognizer mudelit veebikaameraga 
face_cascade = cv2.CascadeClassifier(haar_file) 

cap = cv2.VideoCapture(0)

while True:
	ret, frame = cap.read()
	if ret == False:
		break
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
	faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
	for (x, y, w, h) in faces: 
		cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2) 
		face = gray[y:y + h, x:x + w] 
		face_resize = cv2.resize(face, (width, height)) 
		# Proovib nägu tuvastada
		prediction = model.predict(face_resize) 
		cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3) 

		if prediction[1]<500: 

			cv2.putText(frame, '% s - %.0f' %
			    (names[prediction[0]], prediction[1]), (x-10, y-10), 
			   cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0)) 
		else: 
			cv2.putText(frame, 'tundmatu',(x-10, y-10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0)) 

	cv2.imshow('OpenCV', frame) 

	if cv2.waitKey(1) & 0xFF == ord('q'):
		break
cap.release()
cv2.destroyAllWindows()


Recognizing Face Please Be in sufficient Lights...


1. **Proovige tuvastajate erinevates tingimustes (pime, hämar, profiil jne.)**
2. **Proovige lisada ka kaaslaste nägusid ja treenida nende pealt rohkemate nägude tuvastaja**
3. **Proovige lahendada to-do1 ja to-do2 ning vaadake, kas programm läheb paremaks** 


To-do 1 vihjed: Muuda programmi nii, et see koguks määratud arvu pilte.
1. *Defineeri muutuja*: Määra piltide arv, mida soovid koguda, näiteks `x = 30`. Seejärel asenda `while` tsükli tingimus `count < 30` selle muutujaga.
2. *Näide*:
   ```python
   x = 50  # Näiteks kui soovid koguda 50 pilti
   while count < x:
   ```

To-do 2: Nõua, et iga järgnev pilt oleks eelnevast piisavalt erinev.
1. *Salvesta eelmine pilt*: Hoia viimase salvestatud pildi koopiat, et võrrelda seda järgmise pildiga.
2. *Kasuta eukleidilist kaugust*: Arvuta eukleidiline kaugust kahe järjestikuse pildi vahel, kasutades pildi pikslite väärtusi (või eile arutatud *Linear Binary Pattern Histogram*). Võrdle seda mingi eelnevalt määratud lävendiga, et otsustada, kas salvestada järgmine pilt.

Näiteks, Eukleidiline kaugus kahe punkti $ \mathbf{p} $ ja $ \mathbf{q} $ vahel $ \mathbb{R}^n$ ruumis arvutatakse järgneva valemi järgi:

$$ 
d(\mathbf{p}, \mathbf{q}) = \sqrt{\sum_{i=1}^{n} (p_i - q_i)^2}
$$

See valem näitab, et eukleidiline kaugus on punktide koordinaatide erinevuste ruutude summa ruutjuur.

3. *Pildi vektoriks teisendamine*: Teisenda pilt vektoriks (nt kasutades `.flatten()` meetodit numpy's), mis teeb kauguse arvutamise lihtsamaks.
4. *Näide*:
   ```python
   import numpy as np
   threshold = 1000  # Määra eukleidilise kauguse lävi
   last_face = None  # Salvesta viimane näopilt

   for (x, y, w, h) in faces:
       face = gray[y:y + h, x:x + w]
       face_resize = cv2.resize(face, (width, height))
       current_face = np.array(face_resize).flatten()

       if last_face is not None:
           distance = np.linalg.norm(current_face - last_face)
           if distance < threshold:
               continue  # Jäta see pilt vahele, kui see on liiga sarnane eelmisega

       last_face = current_face  # Uuenda viimast pilti
       cv2.imwrite('%s/%s.png' % (path, count), face_resize)
       count += 1
   ```

Lisaks:
- *Aja järgi salvestamine*: Kui soovid pilte salvestada kindlate ajavahemike järel, võiksid kasutada ajafunktsioone (nt `time.sleep(sec)`).

Üldine nõuanne:
- *Testi ja kohanda lävend*: Eukleidilise kauguse lävendi valimisel võib osutuda vajalikuks katsetamine ja kohandamine, sõltuvalt konkreetsest rakendusest ja kaamerast saadud pildi kvaliteedist.



### 2. variant: Dinomäng käežestidega

Et mängida `chrome://dino/` mängu käežestidega, siis võiks selleks kasutada kolme paketti:`opencv`, `mediapipe` ja `pyautogui`. 

1. **OpenCV veebikaamera tsükkel on antud töövihiku päises** 
2. **Lisa sinna juurde `mediapipega` käetuvastus** vt (https://mediapipe.readthedocs.io/en/latest/solutions/hands.html)
3. **Kui käsi on tuvastatud, siis lisa õige žesti juurde** `pyautogui` käsk, mis vajutab `space`'i
```
#Pane Dino hüppama
pyautogui.press('space')
```

<img src="./kasi.webp" alt="drawing" width="500"/>

In [4]:
import math
import cv2
import pyautogui
import mediapipe as mp

In [13]:
def is_thumbs_up(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    # Kontrolli kas pöial on kõrgemal kui nimetissõrm
    return thumb_tip.y < index_finger_tip.y

mp_hands = mp.solutions.hands #defineerib käed
hands = mp_hands.Hands( #defineerib, mitu kätt tuvastab, ning tuvastuskindluse
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils #defineerib käe skeletoni joonistaja

capture = cv2.VideoCapture(0) #alustab veebikaamera
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640) #madaladab veebikaamera kvaliteeti
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
## Siit alates tuleb lisada juurde punktid 1 - 3
##
##
##


### 3. variant: Koloriseeri (žesti peale)
<img src="./bw-to-color.jpg" alt="drawing" width="500"/>

(kood: https://www.geeksforgeeks.org/black-and-white-image-colorization-with-opencv-and-deep-learning/)

Antud on kolorisaator:

In [7]:
import cv2
from cv2 import dnn

#--------Mudelifailide asukohad--------#
proto_file = './colorization/colorization_deploy_v2.prototxt'  # Prototxt-fail, mis defineerib mudeli kihtide struktuuri
model_file = './colorization/colorization_release_v2.caffemodel'  # Treenitud mudeli fail
hull_pts = 'colorization/pts_in_hull.npy'  
img_path = 'minbw.jpg'  # Mustvalge pildi asukoht
#--------------#--------------#

#--------Loeme mudeli parameetrid--------#
net = dnn.readNetFromCaffe(proto_file, model_file)  # Laadime mudeli Caffe formaadis
kernel = np.load(hull_pts)  # Laadime klastrikeskpunktid
#-----------------------------------#---------------------#

#-----Loeme sisse ja eeltöötleme pildi --------#
img = cv2.imread(img_path)  # Laadime pildi
scaled = img.astype("float32") / 255.0  # Skaleerime piksliväärtused vahemikku 0..1
lab_img = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)  # Teisendame pildi LAB värviruumi
#-----------------------------------#---------------------#

# Lisame mudelile klasterkeskmed 1x1 konvolutsioonidena
class8 = net.getLayerId("class8_ab")  # Leiame kihi ID
conv8 = net.getLayerId("conv8_313_rh")
pts = kernel.transpose().reshape(2, 313, 1, 1)  # Teisendame ja kujundame klastri keskpunktid sobivaks
net.getLayer(class8).blobs = [pts.astype("float32")]  # Määrame kihi parameetrid
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]  # Määrame kihi parameetrid
#-----------------------------------#---------------------#

# Muudame pildi suurust vastavalt võrgu nõuetele
resized = cv2.resize(lab_img, (224, 224))  # Muudame pildi suurust
# Eraldame L kanali
L = cv2.split(resized)[0]  # Eraldame Luminance kanali
# L kanalilt lahutame keskmise
L -= 50  # Keskmise lahutamine
#-----------------------------------#---------------------#

# Prognoosime ab kanaleid sisendiks olevast L kanalist
net.setInput(cv2.dnn.blobFromImage(L))  # Seadistame sisendi
ab_channel = net.forward()[0, :, :, :].transpose((1, 2, 0))  # Saame tulemused ja teisendame need sobivaks
# Suurendame prognoositud 'ab' mahtu sama suureks kui meie sisendpilt
ab_channel = cv2.resize(ab_channel, (img.shape[1], img.shape[0]))

# Võtame pildilt L kanali
L = cv2.split(lab_img)[0]
# Ühendame L kanali prognoositud ab kanaliga
colorized = np.concatenate((L[:, :, np.newaxis], ab_channel), axis=2)

# Teisendame pildi tagasi BGR värviruumi
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)  # Piirame väärtused vahemikku 0..1

# Muudame pildi väärtused vahemikku 0-255 ja teisendame need täisarvuliseks
colorized = (255 * colorized).astype("uint8")

# Muudame piltide suurusi ja kuvame need kõrvuti
img = cv2.resize(img,(640,640))
colorized = cv2.resize(colorized,(640,640))

result = cv2.hconcat([img, colorized])  # Kombineerime pildid kõrvuti

cv2.imshow("Grayscale -> Colour", result)  # Kuvame tulemuse

cv2.waitKey(0)  # Ootame kasutajalt käsku pildi sulgemiseks

-1

1. **Proovige esmalt koloriseerijat mõne enda pildi peal**
2. **Tehke veebikaamera mustvalgeks ja siis koloriseerige *live* pilti**

vihjed: 
* *Kasutage* faili päises olevat veebikaamera tsükklit ning lisage koloriseerija vastavalt sinna sisse (ainult mingi osa koloriseerija koodist tuleb sinna panna, mitte kõike, muidu on kaamera liiga aeglane hiljem)
* *Kaadrite saamine kaamerast*:
    - `capture.read()` kasutamine tagab, et saame kaamerast järgmise kaadri. `ret` on tõeväärtus (True/False), mis näitab, kas kaadri lugemine õnnestus.
   
    ```python
    ret, frame = capture.read()  # Loeb kaamera järgmise kaadri
    ```

* *Kaadri teisendamine halliks ja tagasi BGR-iks*:
    - Esimeses etapis peaks teisendama värvilise veebikaamera BGR kaadri halltoonideks (`cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)`).
    - Seejärel teisendama halltoonides kaader tagasi BGR formaati (`cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)`), et säilitada kolmekanaliline struktuur, mis on vajalik edasisteks operatsioonideks.
    

* *Kaadri skaleerimine ja ettevalmistamine värviliseks muutmiseks*:
    - Skaleerimine, (`frame.astype("float32") / 255.0`), et normaliseerida pikslite väärtused vahemikku 0 kuni 1. See aitab parandada mudeli töödeldavust ja vähendada andmete varieeruvust.
    - Teisendada see edasi LAB värviruumi.

    ```python
    scaled = frame.astype("float32") / 255.0  # Normaliseeri pikslite väärtused
    gray_img = cv2.cvtColor(scaled, cv2.COLOR_BGR2GRAY)  # Teisenda skaalitud kaader halltoonideks
    bgr_from_gray = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)  # Teisenda halltoonides kaader tagasi BGR formaati
    lab_img = cv2.cvtColor(bgr_from_gray, cv2.COLOR_BGR2LAB)  # Teisenda BGR kaader LAB värviruumi
    ```


3. **Lisage mustvalge, koloriseeritud ja originaalne režiim žesti peale**

*Lisa sinna juurde `mediapipega` käetuvastus* vt (https://mediapipe.readthedocs.io/en/latest/solutions/hands.html)
Abiks: on Variant 2 alguses antud kood.
